In [1]:
import numpy as np
import numpy.random
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread, regrid

import matplotlib.pyplot as plt
hv.extension('matplotlib')
hv.extension('bokeh')

In [2]:
%%output backend='matplotlib'
%opts Scatter3D {+framewise}
%opts Overlay {+framewise}

data = np.random.rand(100,3)

def plot_func(elevation, azimuth):
    scatter = hv.Scatter3D(data)
              
    scatter = scatter.opts({'Scatter3D':{'plot': {'fig_size':300, 'azimuth': azimuth, 'elevation':elevation},
                                         'norm': {'framewise':True}}})
    return scatter


holo = hv.HoloMap({(e,a): plot_func(e,a) for e in range(0, 181, 20)
                   for a in range(-90,91,20)}, kdims=['elev','azim'])
holo

In [24]:
%%output backend='matplotlib'

data = np.random.rand(100,3)

def plot_func(elevation, azimuth):
    scatter = hv.Scatter3D(data)
              
    scatter = scatter.opts({'Scatter3D':{'plot': { 'azimuth': azimuth, 'elevation':elevation},
                                         'norm': {'framewise':True}}})
    return scatter

scatter = hv.Scatter3D(data)

dmap = hv.DynamicMap(callback=plot_func, kdims=['elevation', 'azimuth'], cache_size=1)
dmap = dmap.redim.range(elevation=(0,180), azimuth=(-90,90)).opts(norm=dict(framewise=True))
dmap

In [4]:
%%output backend='matplotlib'
%opts Scatter3D {+framewise}
%opts Overlay {+framewise}

data = np.random.randint(0,100,[1000,3])

def plot_func(xmax, ymax):
    scatter = hv.Scatter3D(data[(data[:,0] > xmax) & (data[:,1] > ymax)])
    
    return scatter


dmap = hv.DynamicMap(callback=plot_func, kdims=['xmax', 'ymax'], cache_size=1)
dmap = dmap.redim.range(xmax=(0,100), ymax=(0,100)).opts(norm=dict(framewise=True))
dmap

In [25]:
%%output backend='matplotlib'

from ipywidgets import interact, interactive
from bokeh.io import push_notebook, show
from ipykernel.pylab.backend_inline import flush_figures

renderer = hv.renderer('matplotlib')
plot = (renderer.get_plot(scatter))

data2 = np.random.randint(0,100,[1000,3])

scatter2 = hv.Scatter3D(data2)

renderer = hv.renderer('matplotlib')
plot = (renderer.get_plot(scatter2))

def update(a, e):
    print(a, e)
    plot.elevation = e
    plot.azimuth = a
    plot.update(0)
    print(plot.elevation)
    display(plot)

interactive_plot = interactive(update, a=(-180,180,1), e=(0,180,1))
output = interactive_plot.children[-1]
output.layout.height='400px'
display(interactive_plot)

In [26]:
%%opts Curve [height=200 width=400 aspect=2]
%%opts ErrorBars [height=200 width=400 aspect=2]
%%output backend='matplotlib' size=200

plt.switch_backend('pdf')

arr_a = np.random.rand(10)
arr_b = np.random.rand(10)

nd1 = {}
nd1['one'] = hv.Curve((np.arange(10), arr_a))
nd1['two'] = hv.Curve((np.arange(10)+0.1, arr_b))

nd2 = {}
nd2['one'] = hv.ErrorBars(( np.hstack([np.arange(10)[:,np.newaxis], arr_a[:, np.newaxis], np.random.rand(10,2)])))
nd2['two'] = hv.ErrorBars(( np.hstack([(np.arange(10)+0.1)[:,np.newaxis], arr_b[:, np.newaxis]+0.1, np.random.rand(10,2)])))


ndover1 = hv.NdOverlay(nd1)
ndover2 = hv.NdOverlay(nd2)

ndover1 * ndover2


In [27]:
%%opts Curve {+framewise} [height=200 width=350 aspect=2]
%%opts ErrorBars {+framewise} [height=200 width=350 aspect=2]
%%output backend='bokeh' size=200 holomap='scrubber'

data1 = np.hstack([np.arange(10000)[:,np.newaxis], np.random.rand(10000,3)])
tab1 = hv.Table(data1, kdims='x', vdims=['y','yerr+','yerr-'])
data2 = np.hstack([np.arange(10000)[:,np.newaxis]+0.1, np.random.rand(10000,3)])
tab2 = hv.Table(data2, kdims='x', vdims=['y','yerr+','yerr-'])

def plot(start_time, end_time):
    nd1 = {}
    tab1sel = tab1[0:end_time]
    tab2sel = tab2[0:end_time]
    extents = (start_time, None, end_time, None)
    nd1['one'] = tab1sel.to.curve(kdims=['x'], vdims=['y'], extents=extents)
    nd1['one'].redim(x={'range':(start_time, end_time)})
    nd1['two'] = tab2sel.to.curve(kdims=['x'], vdims=['y'], extents=extents)
    nd1['two'].redim(x={'range':(start_time, end_time)})
    
    nd2 = {}
    nd2['one'] = tab1sel.to.errorbars(kdims=['x'], vdims=['y','yerr+','yerr-'], extents=extents)
    nd2['two'] = tab2sel.to.errorbars(kdims=['x'], vdims=['y','yerr+','yerr-'], extents=extents)


    ndover1 = hv.NdOverlay(nd1)
    ndover1.redim.range(x=(start_time, end_time))
    ndover2 = hv.NdOverlay(nd2)
    ndover2.redim.range(x=(start_time, end_time))
    
    over = ndover1 * ndover2
    over.redim.range(x=(start_time,end_time))
    return over

dmap = hv.DynamicMap(plot, kdims=[hv.Dimension('start_time', range=(0,10000)), hv.Dimension('end_time', range=(0,10000))])

dmap

In [8]:
%%opts Curve {+framewise}
%%output backend='bokeh'

data1 = np.hstack([np.arange(10000)[:,np.newaxis], np.random.rand(10000,3)])
data1 = np.vstack([data1[0:5000], data1[5100:10000]])
tab1 = hv.Table(data1, kdims='x', vdims=['y','yerr+','yerr-'])

def plot(start_time):
    tab1sel = tab1[start_time: start_time+10]
    curv = tab1sel.to.curve(extents=(start_time, None, start_time+10, None))
    
    return curv

dmap = hv.DynamicMap(plot, kdims=[hv.Dimension('start_time', range=(0,10000))])

dmap

In [9]:
import numpy as np
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread
from holoviews.operation import decimate
hv.extension('bokeh')
decimate.max_samples=1000
dynspread.max_px=20
dynspread.threshold=0.5

def random_walk(n, f=5000):
    """Random walk in a 2D space, smoothed with a filter of length f"""
    xs = np.convolve(np.random.normal(0, 0.1, size=n), np.ones(f)/f).cumsum()
    ys = np.convolve(np.random.normal(0, 0.1, size=n), np.ones(f)/f).cumsum()
    xs += 0.1*np.sin(0.1*np.array(range(n-1+f))) # add wobble on x axis
    xs += np.random.normal(0, 0.005, size=n-1+f) # add measurement noise
    ys += np.random.normal(0, 0.005, size=n-1+f)
    return np.column_stack([xs, ys])

def random_cov():
    """Random covariance for use in generating 2D Gaussian distributions"""
    A = np.random.randn(2,2)
    return np.dot(A, A.T)

def time_series(T = 1, N = 100, mu = 0.1, sigma = 0.1, S0 = 20):  
    """Parameterized noisy time series"""
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size = N) 
    W = np.cumsum(W)*np.sqrt(dt) # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W 
    S = S0*np.exp(X) # geometric brownian motion
    return S

In [10]:
np.random.seed(1)
points = hv.Points(np.random.multivariate_normal((0,0), [[0.1, 0.1], [0.1, 1.0]], (1000000,)),label="Points")
paths = hv.Path([0.15*random_walk(100000) for i in range(10)],label="Paths")
decimate(points) + datashade(points) + datashade(paths)


In [11]:
shade(aggregate(paths), normalization='log')

In [12]:
datashade(points, aggregator=ds.reductions.mean('val'))

In [13]:
Y, X = (np.mgrid[0:100, 0:100]-50.)/20.

img = hv.Image(np.sin(X**2+Y**2))

def func(x_range, y_range):
    hv_img1 = hv.Image(img, extents=(-0.3,-0.3,0.3,0.3))
    re1 = shade(regrid(hv_img1, dynamic=False, x_range=x_range, y_range=y_range), dynamic=False)
    re1.extents=(-0.3,-0.3,0.3,0.3)
    return re1

hv_img2 = hv.Image(img, bounds=(-0.5,-0.5,0.5,0.5), extents=(-0.2, -0.2, 0.2, 0.2))
hv_img2.redim(x={'range': (-0.2,0.2)})
#rangexy = hv.streams.RangeXY(source=hv_img1)
re2 = regrid(hv_img2)

hv.DynamicMap(func, streams=[hv.streams.RangeXY()]) + re2

In [14]:
aggregate(hv_img2, aggregator=ds.mean('z'), x_sampling=.01, y_sampling=.01)

In [15]:
%%opts RGB [width=400] {+axiswise}

np.random.seed(3)
kdims=['d1','d2']
num_ks=8

def rand_gauss2d():
    return 100*np.random.multivariate_normal(np.random.randn(2), random_cov(), (100000,))

gaussians = {i: hv.Points(rand_gauss2d(), kdims) for i in range(num_ks)}
lines = {i: hv.Curve(time_series(N=10000, S0=200+np.random.rand())) for i in range(num_ks)}

gaussspread = dynspread(datashade(hv.NdOverlay(gaussians, kdims='k'), aggregator=ds.count_cat('k')))
linespread  = dynspread(datashade(hv.NdOverlay(lines,     kdims='k'), aggregator=ds.count_cat('k')))

gaussspread + linespread

In [16]:
%%opts ErrorBars [show_legend=True] (line_color=Cycle(['#FF0000','#00FF00']))
%%opts NdOverlay [show_legend=True]

error = {}
error['one'] = hv.ErrorBars( np.random.rand(10,3), label='one')
error['two'] = hv.ErrorBars( np.random.rand(10,3), label='two')
error_over = hv.NdOverlay(error)

error_over

In [17]:
def shapes(N, radius=0.5): # Positional keyword arguments are fine
    paths = [hv.Path([[(radius*np.sin(a), radius*np.cos(a)) 
                        for a in np.linspace(-np.pi, np.pi, n+2)]], 
                     extents=(-1,-1,1,1)) 
             for n in range(N,N+3)]
    return hv.Overlay(paths)

holomap = hv.HoloMap({(N,r):shapes(N, r) for N in [3,4,5] for r in [0.5,0.75]},  kdims=['N', 'radius'])
dmap = hv.DynamicMap(shapes, kdims=['N','radius'])
holomap + dmap

In [18]:
dmap.data

In [19]:
hmap = hv.HoloMap(dmap.data, kdims=['N', 'radius'])
hmap

In [20]:
frequencies = [0.5, 0.75, 1.0, 1.25]

def sine_curve(phase, freq):
    xvals = [0.1* i for i in range(100)]
    return hv.Curve((xvals, [np.sin(phase+freq*x) for x in xvals]))

# When run live, this cell's output should match the behavior of the GIF below
dmap = hv.DynamicMap(sine_curve, kdims=['phase', 'frequency'])
dmap.redim.range(phase=(0.5,1)).redim.range(frequency=(0.5,1.25))

In [21]:
dmap.data

In [22]:
hv.HoloMap(dmap)